In [1]:
print("Loading modules, classes and methods...")
import multiprocessing
from unidecode import unidecode
from joblib import Parallel, delayed
import numpy as np
import string
import pandas as pd
import time
import os
import numpy as np
import re
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import torchvision
import pickle
from unicodedata import normalize
from keras.preprocessing.text import Tokenizer
from torchvision import transforms, datasets
from tqdm import tqdm
from time import sleep
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import textclassification as tc

Loading modules, classes and methods...


Using TensorFlow backend.


In [53]:
ordem_classes = ['área_eletricamente_inativa',
 'Bloqueio_de_ramo_direito',
 'Bloqueio_de_ramo_esquerdo',
 'Sobrecarga_ventricular_esquerda_(critérios_de_Romhilt-Estes)',
 'Fibrilação_atrial',
 'Flutter_atrial',
 'Bloqueio_atrioventricular_de_2°_grau_Mobitz_I',
 'Pré-excitação_ventricular_tipo_Wolff-Parkinson-White',
 'Sistema_de_estimulação_cardíaca_normofuncionante',
 'Taquicardia_atrial_multifocal',
 'Taquicardia_supraventricular',
 'Alterações_primárias_da_repolarização_ventricular',
 'Extrassístoles_supraventriculares',
 'Extrassístoles_ventriculares',
 'Bradicardia_sinusal',
 'ECG_dentro_dos_limites_da_normalidade_para_idade_e_sexo',
 'Alterações_inespecíficas_da_repolarização_ventricular',
 'Alterações_secundárias_da_repolarização_ventricular',
 'Arritmia_sinusal',
 'Ausência_de_sinal_eletrocardiográfico_que_impede_a_análise',
 'Possível_inversão_de_posicionamento_de_eletrodos',
 'Bloqueio_atrioventricular_de_1°_grau',
 'Bloqueio_divisional_anterossuperior_do_ramo_esquerdo',
 'Bloqueio_divisional_posteroinferior_do_ramo_esquerdo',
 'Desvio_do_eixo_do_QRS_para_direita',
 'Desvio_do_eixo_do_QRS_para_esquerda',
 'Distúrbio_de_condução_do_ramo_direito',
 'Distúrbio_de_condução_do_ramo_esquerdo',
 'Intervalo_PR_curto',
 'Intervalo_QT_prolongado',
 'Isquemia_subendocárdica',
 'Progressão_lenta_de_R_nas_derivações_precordiais',
 'Ritmo_atrial_ectópico',
 'Sobrecarga_atrial_esquerda',
 'Taquicardia_sinusal']

thresh = np.load("output/nn_thresholds.npy")

In [31]:
db = pd.read_csv("../../data/DATA_LAUDOS_TEXTO_formato1", sep = ";")
db = db[0:10000]
db = db.reset_index()

gz = np.load("output/gZoneIdx.npy")
gz = set(gz)

In [46]:
print("Detecting device...")
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on a GPU :D")
else:
    device = torch.device("cpu")
    print("Running on a CPU :/")

with open("output/parameters", 'r') as f:
    params = f.read().split("\n")
seq_size, TOP_WORDS = [int(a) for a in params]

with open("output/tokenizer_"+str(TOP_WORDS)+".pickle", 'rb') as handle:
    tokenizer = pickle.load(handle)

net = tc.Net(seq_size, TOP_WORDS)
net.load_state_dict(torch.load("output/network_model.pth",  map_location=device))
print(net.eval())

Detecting device...
Running on a GPU :D
Building NN...
Net(
  (l1): Embedding(10000, 128, padding_idx=0)
  (l3): LSTM(128, 128, num_layers=2, dropout=0.2, bidirectional=True)
  (l4): Flatten()
  (l5): Linear(in_features=105216, out_features=35, bias=True)
)


In [47]:
print("Preparing texts...")
text        = [db["CONTEUDO"][i] for i in range(10000) if not (i in gz)]
text        = [tc.clean_text(i) for i in text]
test_X_temp = tokenizer.texts_to_sequences(text)
test_X_temp = [torch.Tensor(i).type(torch.LongTensor) for i in test_X_temp]
test_X_temp = pad_sequence(test_X_temp, batch_first=True).type(torch.LongTensor)

test_X = []
if(len(test_X_temp[0]) < seq_size):
    for i in range(len(test_X_temp)):
        line = test_X_temp[i]
        for j in range(seq_size - len(line)):
            line = torch.cat((line, torch.Tensor([0]).type(torch.LongTensor)), 0)
        test_X.append(line)
else:
    test_X = [line[:seq_size] for line in test_X_temp]
    
test_X = torch.stack([i for i in test_X])

Preparing texts...


In [50]:
print("Passing test through the network...")
finalResult = torch.Tensor()
size = 64
for i in tqdm(range(0, test_X.size()[0], size)):
    with torch.no_grad():
        result      = net(test_X[i: min(i+size, test_X.size()[0])])
    finalResult = torch.cat((finalResult, result), 0)

  0%|          | 0/16 [00:00<?, ?it/s]

Passing test through the network...


100%|██████████| 16/16 [00:27<00:00,  1.70s/it]


In [98]:
a = np.load("output/scores_thresholds.npy")

In [99]:
a

array([80.   , 88.   , 88.   , 90.   , 89.   , 80.   , 85.   , 80.   ,
       62.162, 90.   , 80.   , 90.   , 80.   , 80.   , 85.   , 80.   ,
       78.   , 85.   , 80.   , 75.   , 90.   , 78.   , 80.   , 82.   ,
       90.   , 90.   , 85.   , 82.   , 85.   , 70.   , 80.   , 80.   ,
       80.   , 85.   , 85.   ])

In [97]:
len(thresh)

35

In [83]:
notp = []
predicted = []
for z in range(len(finalResult)):
    sample = finalResult[z]
    ans = np.zeros(35)
    for i in range(35):
        if np.array(sample)[i] > thresh[i]:
            ans[i] = 1
    if ans.sum() == 0:
        notp.append(z)
    else:
        predicted.append([z,ans])

In [89]:
text[15]

'ritmo sinusal.desvio do eixo cardiaco para a esquerda 40 .onda p amplitude normal e segmento negativo em v1 limitrofe.pr duracao normal.qrs duracao normal morfologia normal amplitude normal.st sem supra ou infradesnivelamento significativo.onda t sem alteracoes significativas.qtc 395ms.conclusao1 desvio do eixo cardiaco para a esquerda. dr. fabio torres rabelo crmmg 39287 rede de teleassistencia de minas gerais  belo horizonte o eletrocardiograma e um exame complementar, devendo ser correlacionado aos dados clinicos do paciente. ]] conteudolaudo'

In [85]:
for s in predicted:
    print(text[s[0]], "\n")
    for i in range(len(s[1])):
        if s[1][i]:
            print(ordem_classes[i])
    print()

tracado com interferencia em linha de base, comprometendo a analise precisa do laudo.favor repetir o exame. dr. jorge bezerra cavalcanti sette crmmg51409  telecardio  hc ufmg 

Bloqueio_de_ramo_direito
Possível_inversão_de_posicionamento_de_eletrodos

ritmo sinusal regular.sem desvio de eixo.onda p amplitude e duracao normais.pri duracao diminuida 105msqrs duracao, eixo, morfologia e amplitude normais.st sem supra ou infradesnivelamento.onda t morfologia habitual.qti duracao normal.conclusao1 pr curto. dr. antonio carlos r. cabral crmmg 40.788  telecardio  hc ufmg o eletrocardiograma e um exame complementar que deve ser interpretado juntamente com os dados clinicos do paciente. 

Pré-excitação_ventricular_tipo_Wolff-Parkinson-White
Extrassístoles_ventriculares
Alterações_inespecíficas_da_repolarização_ventricular

ritmo sinusal regular.sem desvio de eixo.onda p amplitude e duracao normais.pri duracao normal.qrs duracao, eixo, morfologia e amplitude normais.st e onda t alteracoes discre